# Set ROIs

This notebook allows you to make ROIs for a video without going through the entire pipeline. The example frame used as the image to set the ROIs is fetched using OpenCV's video reader.\
The functionality of this notebook is technically subsumed by the basic pipeline notebook.

**If your data is local**: Just specify the path to the video you want to draw ROIs on.

**If your data is on a server**: OpenCV's video reader allows for you to read just a single frame from a video file without transferring the entire thing IF the server directory is mounted. So we recommend mounting the server directory if possible to avoid transferring the entire file. This can be done with:
- Windows: https://support.microsoft.com/en-us/windows/map-a-network-drive-in-windows-29ce55d1-34e3-a7e2-4801-131475f9557d
- OSX: https://www.google.com/search?q=mount+network+drive+osx
- Linux: Use RClone CLI: `rclone mount remote:path/to/files /path/to/local/mount`
    - Example: `rclone mount transfer:/n/files/Neurobio/MICROSCOPE/ /mnt/MICROSCOPE/`
    

In [2]:
# ALWAYS RUN THIS CELL
# widen jupyter notebook window
from IPython.display import display, HTML
display(HTML("<style>.container {width:95% !important; }</style>"))

%load_ext autoreload
%autoreload 2
import face_rhythm as fr

from pprint import pprint
from pathlib import Path

import cv2

import numpy as np
import torch
import matplotlib.pyplot
from tqdm import tqdm

fr.util.system_info(verbose=True);

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
== Operating System ==: uname_result(system='Linux', node='rich-L-CLASS', release='5.15.0-76-generic', version='#83~20.04.1-Ubuntu SMP Wed Jun 21 20:23:31 UTC 2023', machine='x86_64')
== CPU Info ==: {'n_cores': 36, 'brand': 'Intel(R) Core(TM) i9-10980XE CPU @ 3.00GHz'}
== RAM ==: svmem(total=269925031936, available=243630243840, percent=9.7, used=23660290048, free=75224793088, active=80180273152, inactive=109870620672, buffers=80977772544, cached=90062176256, shared=332603392, slab=3671760896)
== GPU Info ==: {0: {'id': 0, 'uuid': 'GPU-361d808e-1136-2078-83bf-4290bf948f25', 'load': 0.32, 'memoryUtil': 0.050577799479166664, 'memoryTotal': 24576.0, 'memoryUsed': 1243.0, 'memoryFree': 22983.0, 'driver': '525.125.06', 'name': 'NVIDIA GeForce RTX 3090', 'serial': '1324220011997', 'display_mode': 'Enabled', 'display_active': 'Enabled', 'temperature': 47.0}}
== Conda Environment ==: FR
== Python Version =

## Set paths

In [3]:
directory_videos  = '/mnt/MICROSCOPE/Gyu/BMI_data/cage_0322/'
filename_strMatch = '.*/2023.*cam4.*avi'  ## You can use regular expressions to search and match more complex strings
# filename_strMatch = 'cam4.*avi'  ## You can use regular expressions to search and match more complex strings


In [ ]:
paths_videos = fr.helpers.find_paths(
    dir_outer=directory_videos,
    reMatch=filename_strMatch,  ## string to use to search for files in directory. Uses regular expressions!
    depth=5,  ## how many folders deep to search
)

display(paths_videos)

In [ ]:
directory_save = r'/home/rich/Desktop/0322N_and_0322R/'

## Fetch image

In [ ]:
vc = cv2.VideoCapture(paths_videos[0])
s, image = vc.read()

# Define ROIs

Either select new ROIs (`select_mode='gui'`), or import existing ROIs (`path_file=path_to_ROIs.h5_file`).\
Typically, you should make 1 or 2 ROIs. One for defining where the face points should be and one for cropping the frame.

In [ ]:
# %matplotlib notebook
rois = fr.rois.ROIs(
    select_mode='gui',
    exampleImage=image,
    verbose=2
)

Save the `ROIs` object in the 'analysis_files' project folder

In [ ]:
rois.make_points(rois[0], point_spacing=9)
rois.plot_rois()

In [ ]:
# path_save = str(Path(directory_save) / 'ROIs.h5')
# rois.save_run_data(path_run_data=path_save, overwrite=True, verbose=1)

# Optional: Multisession alignment

The below code shows how to align the points from one 'template' session onto multiple other 'new' sessions.\
Steps:
1. Get example images from each session: `images`
2. Optionally adjust the local contrast of each example image to make: `images_toUse`
3. Instantiate the `ROI_Aligner` class and choose which OpenCV optical flow method to use: `aligner`
4. Perform non-rigid registration to warp the 'template' ROIs and points onto the example images from each session: `aligner.align_and_make_ROIs`
5. Retrieve the newly made `ROIs` class objects: `rois_objs_new = aligner.ROIs_objects_new`
6. Save the new `ROIs` class objects: `rois_objs_new[x].save_run_data()`
7. Visualize the results!

1. Get example images

In [ ]:
import warnings
from tqdm import tqdm

def get_first_frames(paths, max_attempts=5, verbose=True):
    if isinstance(paths, list) == False:
        paths = [paths]

    def get_first_frame(path, attempt=0):
        frame = cv2.VideoCapture(path).read()[1]
        if frame is None:
            print(f"Received `None` for path: {path}. Attempting again; attempt number: {attempt}")
            if attempt + 1 > max_attempts:
                warnings.warn(f"Failed to get first frame for path:{path}")
                return None
            frame = get_first_frame(path, attempt=attempt+1)
        return frame

    flag_singleOutput = len(paths) == 1
    out = [get_first_frame(path) for path in tqdm(paths, disable=flag_singleOutput)]
    out = out[0] if flag_singleOutput else out
    
    return out
        

In [ ]:
images = {path: get_first_frames(path) for path in tqdm(paths_videos)}

2. Optionally adjust local contrast

3. to 5. Register ROIs from template to new session images, then make new `ROIs` objects

In [ ]:
aligner = fr.rois.Image_Aligner(verbose=True)

ims_augmented = aligner.augment_images(
    ims=list(images.values()),
    use_CLAHE=True,
    CLAHE_grid_size=5,
    CLAHE_clipLimit=1,
    CLAHE_normalize=True,
)

In [ ]:
fr.visualization.display_toggle_image_stack(ims_augmented)

In [ ]:
im_template = rois.exampleImage.copy()

aligner.fit_geometric(
    template=im_template,
    ims_moving=ims_augmented,  ## input images
    template_method='image',  ## 'sequential': align images to neighboring images (good for drifting data). 'image': align to a single image
    mode_transform='euclidean',  ## type of geometric transformation. See openCV's cv2.findTransformECC for details
    mask_borders=(0,150,150,50),  ## number of pixels to mask off the edges (top, bottom, left, right)
    n_iter=50,  ## number of iterations for optimization
    termination_eps=1e-09,  ## convergence tolerance
    gaussFiltSize=31,  ## size of gaussian blurring filter applied to all images
    auto_fix_gaussFilt_step=10,  ## increment in gaussFiltSize after a failed optimization
)

aligner.transform_images_geometric(ims_augmented);

im_template_geo = aligner.transform_images(
    ims_moving=[im_template],
    remappingIdx=aligner.remappingIdx_geo,
)[0]

In [ ]:
aligner.fit_nonrigid(
    template=im_template_geo,
#     template=int(0),  ## specifies which image to use as the template. Either array (image), integer (ims_moving index), or float (ims_moving fractional index)
    ims_moving=aligner.ims_registered_geo,  ## Input images. Typically the geometrically registered images
    remappingIdx_init=aligner.remappingIdx_geo,  ## The remappingIdx between the original images (and ROIs) and ims_moving
    template_method='image',  ## 'sequential': align images to neighboring images. 'image': align to a single image, good if using geometric registration first
    mode_transform='createOptFlow_DeepFlow',  ## algorithm for non-rigid transformation. Either 'createOptFlow_DeepFlow' or 'calcOpticalFlowFarneback'. See openCV docs for each. 
    kwargs_mode_transform=None,  ## kwargs for `mode_transform`
)

aligner.transform_images_nonrigid(list(images.values()));

In [ ]:
fr.visualization.display_toggle_image_stack(aligner.ims_registered_geo)

fr.visualization.display_toggle_image_stack(aligner.ims_registered_nonrigid)

In [ ]:
remappingIdx = copy.deepcopy(aligner.remappingIdx_nonrigid)

points_roiBorders_transformed = [{key: aligner.transform_points(
    points=points,
    remappingIdx=rmap_idx,
) for key, points in rois.roi_points.items()} for rmap_idx in remappingIdx]

points_forTracking_transformed = [aligner.transform_points(
    points=rois.point_positions,
    remappingIdx=rmap_idx,
) for rmap_idx in remappingIdx]

exampleImages = list(images.values())

In [ ]:
rois_objs_new = {name: fr.rois.ROIs(
    select_mode='custom', 
    coords_rois=borderPoints, 
    exampleImage=exampleImage, 
    point_positions=point_positions
) for name, borderPoints, exampleImage, point_positions in tqdm(zip(paths_videos, points_roiBorders_transformed, exampleImages, points_forTracking_transformed))}

## 6. Save
Save the new `ROIs` objects. These can be used to initialize the `ROIs` objects in each face-rhythm run.

In [ ]:
# names_sessions = ['__'.join(Path(name).parts[-6:-4]) for name in rois_objs_new.keys()]

for ii, (name, rois_new) in enumerate(rois_objs_new.items()):
    path_save = str(Path(directory_save) / names_sessions[ii] / f'ROIs.h5')
#     path_save = str(Path(directory_save) / f'ROIs.h5')
    rois_new.save_run_data(
        path_run_data=path_save,
        overwrite=True,
        verbose=1,
    )

## 7. Visualize!

In [ ]:
fr.visualization.display_toggle_image_stack(aligner.ims_registered_nonrigid)

In [ ]:
## Cast to uint8
movie = [im.astype(np.uint8) for im in aligner.ims_registered_nonrigid]
## Add text overlay
movie = fr.helpers.add_text_to_images(movie, [[str(n)] for n in np.arange(len(movie))], position=(50,100), font_size=4, line_width=5,)

image_saver = fr.util.Image_Saver(
    dir_save=directory_save,
    overwrite=True,
)

image_saver.save_gif(
    array_images=movie, 
    name_save='mouse_face_matched', 
    frame_rate=10.0, 
    loop=0, 
)

In [ ]:
frame_visualizer = fr.visualization.FrameVisualizer(
    display=False,
    frame_height_width=rois_objs_new[list(rois_objs_new.keys())[0]].img_hw,
    point_sizes=3,
    points_colors=(255,0,0),
    alpha=0.5,
)

images_with_warped_points = [frame_visualizer.visualize_image_with_points(
    image=rois.exampleImage,
    points=rois.point_positions,
) for rois in rois_objs_new.values()]

In [ ]:
fr.visualization.display_toggle_image_stack(images_with_warped_points)

In [ ]:
## Cast to uint8
movie = [im.astype(np.uint8) for im in images_with_warped_points]
## Add text overlay
movie = fr.helpers.add_text_to_images(movie, [[str(n)] for n in np.arange(len(movie))], position=(50,100), font_size=4, line_width=5,)

image_saver = fr.util.Image_Saver(
    dir_save=directory_save,
    overwrite=True,
)

image_saver.save_gif(
    array_images=movie, 
    name_save='mouse_face_points_matched', 
    frame_rate=3.0, 
    loop=0, 
)